In [1]:
from selenium import webdriver
import json
import time
import pandas as pd
pd.options.display.max_columns = None
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from IPython.display import display, clear_output
import random
import urllib
import math
from operator import itemgetter

In [2]:
driver = webdriver.Chrome('./chromedriver')

## Сбор ссылок моделей

In [3]:
driver.get('https://auto.ru/catalog/cars/all/?view_type=list&year_from=2019&year_to=2019')
models_links = []
next_button = driver.find_element_by_class_name('pager__next')

while len(models_links) < 10:
    time.sleep(1)
    elem = driver.find_element_by_class_name('catalog-all-text-list')
    models_links+= [x.get_attribute('href') for x in elem.find_elements_by_tag_name('a')]
    time.sleep(1)
    if next_button.is_enabled():
        next_button.click()
    else:
        break
    
with open('models_links.json', 'w+') as out_file:
    json.dump({'links': models_links}, out_file)

In [4]:
models_links

['https://auto.ru/catalog/cars/acura/ilx/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/acura/mdx/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/acura/nsx/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/acura/rdx/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/acura/rlx/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/acura/tlx/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/alfa_romeo/giulia/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/alfa_romeo/giulietta/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/alfa_romeo/stelvio/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/alpina/b3/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/alpina/b4/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/alpina/b5/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/alpina/b7/?year_from=2019&year_to=2019',
 'https://auto.ru/catalog/cars/alpin

## Сбор ссылок конфигураций всех моделей

In [5]:
all_models_links = []
for models_link in models_links[:1]:
    driver.get(models_link)
    elem = driver.find_elements_by_class_name('catalog-all-text-list__desc')
    for e in elem:
        models_types = e.find_elements_by_class_name('mosaic__title')
        for model_type in models_types:
            all_models_links.append(model_type.get_attribute('href'))
with open('all_models_links.json', 'w') as out_file:
    json.dump({'links': all_models_links}, out_file)

In [6]:
all_models_links

['https://auto.ru/catalog/cars/acura/ilx/21468782/21468837/?year_from=2019&year_to=2019']

## Сбор характеристик

In [7]:
data = pd.DataFrame()
for link in all_models_links:
    driver.get(link)
    time.sleep(random.randint(3, 7))


    wait = WebDriverWait(driver, 60)


    brand = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_mark').text
    model = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_model').text
    generation = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_generation').text
    config = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_configuration').text

    try:
        used_price_string = driver.find_element_by_xpath("//div[@class='catalog-generation__sales-info-block'][div[@class='catalog-generation__sales-info-block-title']='С пробегом']/div[@class='catalog-generation__sales-info-block-link']/a").text
        price = (int(a.split('–')[0].replace(' ','')) + int(a.split('–')[1].replace(' ','').replace('₽','')))/2
    except:
        price = None

    button = wait.until(ec.element_to_be_clickable((By.LINK_TEXT, 'Характеристики')))
    ActionChains(driver).click(button).perform()

    wait.until(ec.presence_of_element_located((By.XPATH, "//dt[text()='Тип двигателя']")))
    time.sleep(random.randint(5, 7))


    models_repres = driver.find_elements_by_css_selector(".catalog-table__inner .link_theme_auto")

    for i in range(len(models_repres)):
        wait.until(ec.presence_of_element_located((By.XPATH, "//dt[text()='Тип двигателя']")))
        time.sleep(random.randint(5, 7))
        driver.find_elements_by_css_selector(".catalog-table__inner .link_theme_auto")[i].click()
        #считываем характеристики
        spec_dict={}
        spec_dict['Марка'] = brand
        spec_dict['Модель'] = model
        spec_dict['Поколение'] = generation
        spec_dict['Тип кузова'] = config
        spec_dict['Цена'] = price


        wait.until(ec.presence_of_element_located((By.XPATH, "//dt[text()='Тип двигателя']")))
        time.sleep(random.randint(5, 7))
        for group in driver.find_elements_by_class_name('catalog__details-group'):
            a = group.find_elements_by_class_name('list-values__label')
            b = group.find_elements_by_class_name('list-values__value')
            for x, y in zip(a,b):
                spec_dict[x.text] = [y.text]


        comp = driver.find_elements_by_link_text('Комплектации')
        try:
            if len(comp) != 0:
                time.sleep(5)
                comp[0].click()
                time.sleep(5)
                spec_dict['Цена'] = int(driver.find_element_by_class_name('catalog__package-price').text.replace(' ','').replace('₽',''))
        except:
            pass

        button = wait.until(ec.element_to_be_clickable((By.LINK_TEXT, 'Характеристики')))
        ActionChains(driver).click(button).perform()

        spec_dict['Ссылка'] = link
        temp_data = pd.DataFrame.from_dict(spec_dict)
        data = data.append(temp_data, sort=False)
        data.to_csv('out.csv', index=False)
        print(link)
data = data.reset_index().drop(columns=['index']).reset_index()

https://auto.ru/catalog/cars/acura/ilx/21468782/21468837/?year_from=2019&year_to=2019


## Сбор изображений

In [8]:
data['Изображение'] = ''
for index in range(data.shape[0]):
    if index != 0:
        if data.at[index-1, 'Ссылка'] == data.at[index, 'Ссылка']:
            data.at[index, 'Изображение'] = data.at[index-1, 'Изображение']
        else:
            driver.get(data.at[index, 'Ссылка'])
            driver.find_element_by_class_name('photo-gallery__thumb').click()
            src = driver.find_element_by_class_name('fotorama__img').get_attribute('src')
            urllib.request.urlretrieve(src, "./images/"+str(index)+'.png')
    else:
        driver.get(data.at[index, 'Ссылка'])
        driver.find_element_by_class_name('photo-gallery__thumb').click()
        src = driver.find_element_by_class_name('fotorama__img').get_attribute('src')
        urllib.request.urlretrieve(src, "./images/"+str(index)+'.png')

In [9]:
data

,index,Марка,Модель,Поколение,Тип кузова,Цена,Страна марки,Класс автомобиля,Количество дверей,Количество мест,Длина,Ширина,Высота,Колёсная база,Клиренс,Ширина передней колеи,Ширина задней колеи,Размер колёс,"Объем багажника мин/макс, л","Объём топливного бака, л","Снаряженная масса, кг",Коробка передач,Количество передач,Тип привода,Тип передней подвески,Тип задней подвески,Передние тормоза,Задние тормоза,"Разгон до 100 км/ч, с","Расход топлива, л город/трасса/смешанный",Марка топлива,Тип двигателя,Расположение двигателя,"Объем двигателя, см³",Тип наддува,"Максимальная мощность, л.с./кВт при об/мин","Максимальный крутящий момент, Н*м при об/мин",Расположение цилиндров,Количество цилиндров,Число клапанов на цилиндр,Система питания двигателя,Степень сжатия,"Диаметр цилиндра и ход поршня, мм",Ссылка,Изображение
0,0,Acura,ILX,I Рестайлинг 2 2018 – н.в.,Седан,None,Япония,D,4,5,4628,1793,1412,2670,135,1509,1529,215/45/R17 225/40/R18,-,50,1405,робот,8,передний,"независимая, пружинная","независимая, пружинная",дисковые вентилируемые,дисковые,7.7,9.8/6.9/8.4,АИ-92,бензин,"переднее, поперечное",2354,нет,201 / 148 при 6800,245 при 3600,рядное,4,4,распределенный впрыск (многоточечный),11.6,87 × 99,https://auto.ru/catalog/cars/acura/ilx/2146878...,


После этого произвел обработку полученных данных. Например, объем багажника и расход топливо привел к единому виду

Далее важные числовые показатели перевел в качественные характеристики. 

## Пример нахождения расстояний для рекомендации

In [10]:
def eucl_dist(x1, x2):
    return sum([(x1i - x2i) ** 2 for x1i, x2i in zip (x1, x2)])

def manh_dist(x1, x2):
    return sum(abs(x1i - x2i) for x1i, x2i in zip(x1, x2))            

def recommend(data, car_modificaton, columns_for_dist, mode):
    neighbors_list = []
    for index in range(data.shape[0]):
        data_train_current_x = [x for x in data[columns_for_dist].iloc[index]]
        if mode == 'eucl':
            dist = eucl_dist([x for x in car_modificaton[columns_for_dist]], data_train_current_x)
        elif mode == 'manh':
            dist = manh_dist([x for x in car_modificaton[columns_for_dist]], data_train_current_x)
        temp_res = (dist, index, data_train_current_x)
        neighbors_list.append(temp_res)    
    
    neighbors_list_sorted = sorted(neighbors_list, key=itemgetter(0))
    
    real_neighbors_list = []
    already_added = []
    
    for x in neighbors_list_sorted:
        if not data['Марка'].iloc[x[1]] == car_modificaton['Марка'] and not data['Модель'].iloc[x[1]] == car_modificaton['Модель'] and (data['Марка'].iloc[x[1]], data['Модель'].iloc[x[1]]) not in already_added:
            real_neighbors_list.append((x[1], x[0]))
            already_added.append((data['Марка'].iloc[x[1]], data['Модель'].iloc[x[1]]))
        if len(real_neighbors_list) == 5:
            break
    return real_neighbors_list

In [11]:
columns = [ "knn_цена", "knn_длина", "knn_ширина","knn_высота", "knn_бак","knn_скорость","knn_разгон","knn_выбросы",
"knn_двигатель","knn_клиренс", "knn_багажник","knn_расход", "knn_наддув_компрессор","knn_наддув_нет", 
"knn_наддув_турбонаддув","knn_страна_Австрия", "knn_страна_Великобритания","knn_страна_Германия", "knn_страна_Дания",
"knn_страна_Израиль", "knn_страна_Индия","knn_страна_Иран", "knn_страна_Испания","knn_страна_Италия",
"knn_страна_Китай","knn_страна_Малайзия", "knn_страна_Нидерланды","knn_страна_Объединённые Арабские Эмираты",
"knn_страна_Россия","knn_страна_Румыния", "knn_страна_США", "knn_страна_Тайвань","knn_страна_Узбекистан", 
"knn_страна_Франция","knn_страна_Хорватия", "knn_страна_Чехия","knn_страна_Швеция", "knn_страна_Южная Корея", 
"knn_страна_Япония", "knn_коробка_автомат", "knn_коробка_вариатор", "knn_коробка_механика", "knn_коробка_робот",
"knn_привод_задний", "knn_привод_передний", "knn_привод_полный", "knn_тип_двигателя_СУГ",
"knn_тип_двигателя_бензин", "knn_тип_двигателя_гибрид", "knn_тип_двигателя_дизель", "knn_тип_двигателя_электро",
"knn_класс_A", "knn_класс_B", "knn_класс_C", "knn_класс_D", "knn_класс_E", "knn_класс_F", "knn_класс_J", 
"knn_класс_M", "knn_класс_S", "knn_эко_Euro 3", "knn_эко_Euro 4", "knn_эко_Euro 5", "knn_эко_Euro 6",
"Количество дверей", "Количество мест"]

full_data = pd.read_csv('data_final.csv')

In [12]:
data = pd.read_csv('data_final.csv')

In [13]:
for index in range(data.shape[0])[554:555]:
    print(data.at[index, 'Марка'], data.at[index, 'Модель'], data.at[index, 'Тип кузова'])
    print(100*'-')
    print('{:>50}|{:>50}'.format('Евклидово расстояние', 'Манхэттенское расстояние'))
    print(100*'-')
    eucl_recommend = recommend(data, data.iloc[index], columns, 'eucl')
    manh_recommend = recommend(data, data.iloc[index], columns, 'manh')
    for x, y in zip(eucl_recommend, manh_recommend):
        print('{:>50}|{:>50}'.format(str(x[1])+' '+str(data.at[x[0], 'Марка'])+' '+str(data.at[x[0], 'Модель'])+' '+str(data.at[x[0], 'Тип кузова']), str(y[1])+' '+str(data.at[y[0], 'Марка'])+' '+str(data.at[y[0], 'Модель'])+' '+str(data.at[y[0], 'Тип кузова'])))
    print(100*'_')

Audi S5 Кабриолет
----------------------------------------------------------------------------------------------------
                              Евклидово расстояние|                          Манхэттенское расстояние
----------------------------------------------------------------------------------------------------
                                0.0 Alpina B4 Купе|                                0.0 Alpina B4 Купе
           2.0 Mercedes-Benz C-Класс AMG Кабриолет|           2.0 Mercedes-Benz C-Класс AMG Кабриолет
               3.0 Mercedes-Benz C-Класс Кабриолет|                         3.0 BMW 4 серия Кабриолет
                              4.0 BMW 4 серия Купе|               3.0 Mercedes-Benz C-Класс Кабриолет
                              5.0 BMW 8 серия Купе|                               5.0 Alpina B3 Седан
____________________________________________________________________________________________________
